In [1]:
import findspark
findspark.init()

import pyspark
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .master("local[*]") \
    .appName('test') \
    .getOrCreate()

## GroupBy

In [2]:
df_green = spark.read.parquet('data/pq/green/*/*')

In [3]:
df_green.createOrReplaceTempView('green')

In [4]:
df_green_revenue = spark.sql("""
SELECT 

    date_trunc('hour', lpep_pickup_datetime) AS hour, 
    PULocationID AS zone,

    SUM(total_amount) AS amount,
    COUNT(1) AS number_records

FROM
    green
WHERE 
    lpep_pickup_datetime >= '2020-01-01 00:00:00'
GROUP BY 
    1, 2
ORDER BY 
    1, 2
""")

In [5]:
df_green_revenue.show()

+-------------------+----+------------------+--------------+
|               hour|zone|            amount|number_records|
+-------------------+----+------------------+--------------+
|2020-01-01 00:00:00|   7| 769.7299999999996|            45|
|2020-01-01 00:00:00|  17|195.03000000000003|             9|
|2020-01-01 00:00:00|  18|               7.8|             1|
|2020-01-01 00:00:00|  22|              15.8|             1|
|2020-01-01 00:00:00|  24|              87.6|             3|
|2020-01-01 00:00:00|  25| 531.0000000000002|            26|
|2020-01-01 00:00:00|  29|              61.3|             1|
|2020-01-01 00:00:00|  32| 68.94999999999999|             2|
|2020-01-01 00:00:00|  33|317.27000000000004|            11|
|2020-01-01 00:00:00|  35|            129.96|             5|
|2020-01-01 00:00:00|  36|295.34000000000003|            11|
|2020-01-01 00:00:00|  37|            175.67|             6|
|2020-01-01 00:00:00|  38| 98.78999999999999|             2|
|2020-01-01 00:00:00|  4

In [6]:
df_green_revenue \
    .repartition(20) \
    .write.parquet('data/report/revenue/green', mode='overwrite')

doing the same thing with yellow data

In [7]:
df_yellow = spark.read.parquet('data/pq/yellow/*/*')

In [8]:
df_yellow.createOrReplaceTempView('yellow')

In [9]:
df_yellow.printSchema()

root
 |-- VendorID: integer (nullable = true)
 |-- tpep_pickup_datetime: timestamp (nullable = true)
 |-- tpep_dropoff_datetime: timestamp (nullable = true)
 |-- passenger_count: integer (nullable = true)
 |-- trip_distance: double (nullable = true)
 |-- RatecodeID: integer (nullable = true)
 |-- store_and_fwd_flag: string (nullable = true)
 |-- PULocationID: integer (nullable = true)
 |-- DOLocationID: integer (nullable = true)
 |-- payment_type: integer (nullable = true)
 |-- fare_amount: double (nullable = true)
 |-- extra: double (nullable = true)
 |-- mta_tax: double (nullable = true)
 |-- tip_amount: double (nullable = true)
 |-- tolls_amount: double (nullable = true)
 |-- improvement_surcharge: double (nullable = true)
 |-- total_amount: double (nullable = true)
 |-- congestion_surcharge: double (nullable = true)



In [10]:
df_yellow_revenue = spark.sql("""
SELECT 

    date_trunc('hour', tpep_pickup_datetime) AS hour, 
    PULocationID AS zone,

    SUM(total_amount) AS amount,
    COUNT(1) AS number_records

FROM
    yellow
WHERE 
    tpep_pickup_datetime >= '2020-01-01 00:00:00'
GROUP BY 
    1, 2
ORDER BY 
    1, 2
""")

In [11]:
df_yellow_revenue.show()

+-------------------+----+------------------+--------------+
|               hour|zone|            amount|number_records|
+-------------------+----+------------------+--------------+
|2020-01-01 00:00:00|   3|              25.0|             1|
|2020-01-01 00:00:00|   4|1004.3000000000002|            57|
|2020-01-01 00:00:00|   7| 455.1700000000001|            38|
|2020-01-01 00:00:00|  10|             42.41|             2|
|2020-01-01 00:00:00|  12|             107.0|             6|
|2020-01-01 00:00:00|  13|1214.8000000000002|            56|
|2020-01-01 00:00:00|  14|               8.8|             1|
|2020-01-01 00:00:00|  15|             34.09|             1|
|2020-01-01 00:00:00|  17|220.20999999999998|             8|
|2020-01-01 00:00:00|  18|               5.8|             1|
|2020-01-01 00:00:00|  24| 754.9500000000002|            45|
|2020-01-01 00:00:00|  25|            324.35|            16|
|2020-01-01 00:00:00|  32|              18.0|             1|
|2020-01-01 00:00:00|  3

In [12]:
df_yellow_revenue \
    .repartition(20) \
    .write.parquet('data/report/revenue/yellow', mode='overwrite')

## Joins

In [13]:
df_green_revenue_tmp = df_green_revenue \
    .withColumnRenamed('amount', 'green_amount') \
    .withColumnRenamed('number_records', 'green_number_records')

df_yellow_revenue_tmp = df_yellow_revenue \
    .withColumnRenamed('amount', 'yellow_amount') \
    .withColumnRenamed('number_records', 'yellow_number_records')

In [14]:
df_join = df_green_revenue_tmp.join(df_yellow_revenue_tmp, on=['hour', 'zone'], how='outer')

In [16]:
df_join.write.parquet('data/report/revenue/total', mode='overwrite')

In [17]:
df_join.show()

+-------------------+----+------------------+--------------------+------------------+---------------------+
|               hour|zone|      green_amount|green_number_records|     yellow_amount|yellow_number_records|
+-------------------+----+------------------+--------------------+------------------+---------------------+
|2020-01-01 00:00:00|   3|              null|                null|              25.0|                    1|
|2020-01-01 00:00:00|   4|              null|                null|1004.3000000000002|                   57|
|2020-01-01 00:00:00|   7| 769.7299999999996|                  45| 455.1700000000001|                   38|
|2020-01-01 00:00:00|  12|              null|                null|             107.0|                    6|
|2020-01-01 00:00:00|  37|            175.67|                   6|161.60999999999999|                    7|
|2020-01-01 00:00:00|  40|168.97999999999996|                   8|             89.97|                    5|
|2020-01-01 00:00:00|  45|  

### Joining df with lesser dataframe

In [18]:
df_join = spark.read.parquet('data/report/revenue/total')

In [19]:
df_join.show()

+-------------------+----+------------------+--------------------+------------------+---------------------+
|               hour|zone|      green_amount|green_number_records|     yellow_amount|yellow_number_records|
+-------------------+----+------------------+--------------------+------------------+---------------------+
|2020-01-01 00:00:00|  14|              null|                null|               8.8|                    1|
|2020-01-01 00:00:00|  15|              null|                null|             34.09|                    1|
|2020-01-01 00:00:00|  17|195.03000000000003|                   9|220.20999999999998|                    8|
|2020-01-01 00:00:00|  25| 531.0000000000002|                  26|            324.35|                   16|
|2020-01-01 00:00:00|  32| 68.94999999999999|                   2|              18.0|                    1|
|2020-01-01 00:00:00|  43|            107.52|                   6| 6539.510000000012|                  390|
|2020-01-01 00:00:00|  49|26

In [20]:
df_zones = spark.read.parquet('zones')

In [21]:
df_zones.show()

+----------+-------------+--------------------+------------+
|LocationID|      Borough|                Zone|service_zone|
+----------+-------------+--------------------+------------+
|         1|          EWR|      Newark Airport|         EWR|
|         2|       Queens|         Jamaica Bay|   Boro Zone|
|         3|        Bronx|Allerton/Pelham G...|   Boro Zone|
|         4|    Manhattan|       Alphabet City| Yellow Zone|
|         5|Staten Island|       Arden Heights|   Boro Zone|
|         6|Staten Island|Arrochar/Fort Wad...|   Boro Zone|
|         7|       Queens|             Astoria|   Boro Zone|
|         8|       Queens|        Astoria Park|   Boro Zone|
|         9|       Queens|          Auburndale|   Boro Zone|
|        10|       Queens|        Baisley Park|   Boro Zone|
|        11|     Brooklyn|          Bath Beach|   Boro Zone|
|        12|    Manhattan|        Battery Park| Yellow Zone|
|        13|    Manhattan|   Battery Park City| Yellow Zone|
|        14|     Brookly

In [22]:
df_result = df_join.join(df_zones, df_join.zone == df_zones.LocationID)

In [27]:
df_result.drop('LocationID', 'zone').write.parquet('tmp/revenue-zone')